In [1]:
using Revise
using Bilevel

using RigidBodyDynamics
using MeshCatMechanisms
using MeshCat
using Plots
using LinearAlgebra
using StaticArrays 

gr(size=(800,500),html_output_format=:png,
    seriescolor=:magma,background_color_legend=:white,
    background=RGB(((240,240,240) ./ 255.)...));

┌ Info: Precompiling Bilevel [9688c538-179f-11e9-3174-495cea6b7f67]
└ @ Base loading.jl:1186
┌ Info: Recompiling stale cache file /home/blandry/.julia/compiled/v1.1/ForwardDiff/k0ETY.ji for ForwardDiff [f6369f11-7733-5829-9624-2563aa707210]
└ @ Base loading.jl:1184
┌ Info: Precompiling GeometryTypes [4d00f742-c7ba-57c2-abde-4428a4b178cb]
└ @ Base loading.jl:1186
┌ Info: Recompiling stale cache file /home/blandry/.julia/compiled/v1.1/RigidBodyDynamics/WeevQ.ji for RigidBodyDynamics [366cf18f-59d5-5db9-a4de-86a9f6786172]
└ @ Base loading.jl:1184
┌ Info: Precompiling MechanismGeometries [931e9471-e8fb-5385-a477-07ad12718aca]
└ @ Base loading.jl:1186
┌ Info: Precompiling MeshCatMechanisms [6ad125db-dd91-5488-b820-c1df6aab299d]
└ @ Base loading.jl:1186
┌ Error: evaluation error
│   mod = Bilevel
│   ex = const usrfun = #= /home/blandry/.julia/dev/Bilevel/src/solvers/snopt.jl:154 =# @cfunction(objcon_wrapper, Cvoid, (Ptr{Clong}, Ref{Clong}, Ptr{Cdouble}, Ref{Clong}, Ref{Clong}, Ptr{Cdouble},

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/b464848323e0a4c9a3cfe11f8edca2932188b6d9-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/b464848323e0a4c9a3cfe11f8edca2932188b6d9-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1186


# Contact

## Single point with contact

In [ ]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)

floor = findbody(mechanism, "floor")
point = Point3D(default_frame(floor), SVector([0.,0.,0.]...))
normal = FreeVector3D(default_frame(floor), SVector([0.,0.,1.]...))
floor_obs = Obstacle(floor, point, normal, :xyz, 1.)

obstacles = [floor_obs]
env = Environment(mechanism, urdf, obstacles)

In [ ]:
q1 = [1., 0., 0., 0., 0., 0., 0.]
qN = [1., 0., 0., 0., .004, 0., 0.]
N = 10
Δt = .005

q_min = [1., 0., 0., 0., -1., -1., -1.]
q_max = [1., 0., 0., 0., 1., 1., 1.]

function get_prob_limits(sim_data)
    x_min = -1e19*ones(sim_data.vs.num_vars)
    x_max = 1e19*ones(sim_data.vs.num_vars)
    
    # joint limits
    add_box_con_snopt!(x_min, x_max, sim_data, :q, q_min, q_max, 1:N)
    add_box_con_snopt!(x_min, x_max, sim_data, :h, [Δt], [Δt], 1:N-1)
    
    # torques limits
    add_box_con_snopt!(x_min, x_max, sim_data, :u, zeros(num_velocities(mechanism)), zeros(num_velocities(mechanism)), 1:N-1)

    x_min, x_max
end

function add_prob_constraints!(sim_data)
    vs = sim_data.vs

    add_eq!(sim_data, :cq1, length(q1), x -> vs(x, :q1) - q1)
    add_eq!(sim_data, :cqN, length(qN), x -> vs(x, Symbol("q", N)) - qN)
    add_eq!(sim_data, :cv1, 1, x -> vs(x, :v1)[6])
    add_eq!(sim_data, :cvN, 1, x -> vs(x, Symbol("v", N))[4])
end

In [ ]:
sim_data = get_trajopt_data_indirect(mechanism,env,Δt,N,relax_comp=false)
add_prob_constraints!(sim_data)
x_min, x_max = get_prob_limits(sim_data)
traj_indirect = Bilevel.trajopt(sim_data, x_min=x_min, x_max=x_max, quaternion_state=true, verbose=1)

In [ ]:
sim_data = get_trajopt_data_semidirect(mechanism,env,Δt,N,relax_comp=false)
add_prob_constraints!(sim_data)
x_min, x_max = get_prob_limits(sim_data)
traj_semidirect = Bilevel.trajopt(sim_data, x_min=x_min, x_max=x_max, quaternion_state=true, verbose=1)

In [ ]:
j = 5
plot(traj_indirect[7],traj_indirect[8][j,:],label="Indirect",seriescolor=RGB(((227,74,51) ./ 255.)...),linewidth=4,marker=6)
plot!(traj_semidirect[7],traj_semidirect[8][j,:],label="Semidirect",seriescolor=RGB(((254,232,200) ./ 255.)...),linewidth=0,marker=4)

# Box with four contact

In [ ]:
μ_floor = .1

urdf = joinpath("..", "urdf", "box.urdf")
mechanism = parse_urdf(Float64, urdf)

floor = findbody(mechanism, "floor")
point = Point3D(default_frame(floor), SVector([0.,0.,0.]...))
normal = FreeVector3D(default_frame(floor), SVector([0.,0.,1.]...))
floor_obs = Obstacle(floor, point, normal, :xyz, μ_floor)

obstacles = [floor_obs]
env = Environment(mechanism, urdf, obstacles);

In [ ]:
Δt = 0.1
N = 10

x0 = MechanismState(mechanism)
set_configuration!(x0,[1.,0.,0.,0.,0.,0.,0.25])
set_velocity!(x0,[0.,0.,0.,.75,0.,0.])
setdirty!(x0)

sim_data = get_sim_data_indirect(mechanism,env,Δt)

null_ctrl! = (u,t,x) -> u[:] .= 0.
traj_sim = Bilevel.simulate(sim_data,null_ctrl!,x0,N)

In [ ]:
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf))
open(mvis)

In [ ]:
setanimation!(mvis, traj_sim[6], traj_sim[1])

In [ ]:
plot(traj_sim[7][5,:])

In [ ]:
q1 = [1., 0., 0., 0., 0., 0., 0.25]
qN = [1., 0., 0., 0., .25, 0., 0.25]
N = 10
Δt = 0.1

q_min = [1., 0., 0., 0., -5., -5., -5.]
q_max = [1., 0., 0., 0., 5., 5., 5.]

function get_prob_limits(sim_data)
    x_min = -1e19*ones(sim_data.vs.num_vars)
    x_max = 1e19*ones(sim_data.vs.num_vars)
    
    # joint limits
    add_box_con_snopt!(x_min, x_max, sim_data, :q, q_min, q_max, 1:N)
    add_box_con_snopt!(x_min, x_max, sim_data, :h, [.9 * Δt], [1.1 * Δt], 1:N-1)
    
    # torques limits
    add_box_con_snopt!(x_min, x_max, sim_data, :u, zeros(num_velocities(mechanism)), zeros(num_velocities(mechanism)), 1:N-1)

    x_min, x_max
end

function get_prob_init(sim_data)
    x0 = zeros(sim_data.vs.num_vars)
    for n = 1:N
       x0[sim_data.vs(Symbol("q", n))] = n/N * (qN - q1) + q1 
    end
end

function add_prob_constraints!(sim_data)
    vs = sim_data.vs

    add_eq!(sim_data, :cq1, length(q1), x -> vs(x, :q1) - q1)
    add_eq!(sim_data, :cqN, length(qN), x -> vs(x, Symbol("q", N)) - qN)
    add_eq!(sim_data, :cv1, 1, x -> vs(x, :v1)[6])
#     add_eq!(sim_data, :cvN, 1, x -> vs(x, Symbol("v", N))[4])
end

In [ ]:
sim_data = get_trajopt_data_indirect(mechanism,env,Δt,N,relax_comp=false)
add_prob_constraints!(sim_data)
x_min, x_max = get_prob_limits(sim_data)
x0 = get_prob_init(sim_data)
traj_indirect = Bilevel.trajopt(sim_data, x0=x0, x_min=x_min, x_max=x_max, quaternion_state=true, verbose=1)

In [ ]:
sim_data = get_trajopt_data_semidirect(mechanism,env,Δt,N,relax_comp=false)
add_prob_constraints!(sim_data)
x_min, x_max = get_prob_limits(sim_data)
x0 = get_prob_init(sim_data)
traj_semidirect = Bilevel.trajopt(sim_data, x0=x0, x_min=x_min, x_max=x_max, quaternion_state=true, verbose=1)

In [ ]:
j = 5
plot(traj_indirect[7],traj_indirect[8][j,:],label="Indirect",seriescolor=RGB(((227,74,51) ./ 255.)...),linewidth=4,marker=6)
plot!(traj_semidirect[7],traj_semidirect[8][j,:],label="Semidirect",seriescolor=RGB(((254,232,200) ./ 255.)...),linewidth=0,marker=4)

## Little dog

In [2]:
urdf = joinpath("..", "urdf", "little_dog", "little_dog_2d.urdf")
mechanism = parse_urdf(Float64, urdf)

floor = findbody(mechanism, "floor")
point = Point3D(default_frame(floor), SVector([0.,0.,0.]...))
normal = FreeVector3D(default_frame(floor), SVector([0.,0.,1.]...))
floor_obs = Obstacle(floor, point, normal, :xyz, 1.)

obstacles = [floor_obs]
env = Environment(mechanism, urdf, obstacles);

In [ ]:
N = 10
Δt = 0.1
q0 = [0., -.175, 0., 0., 0., 0., 0., .036, 0., 0., 0., .00758, 0., .002445, 0.]  
v0 = zeros(num_velocities(mechanism))

sim_data = get_sim_data_indirect(mechanism,env,Δt)
null_ctrl! = (u,t,x) -> u[:] .= 0.
x0 = MechanismState(mechanism)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
traj = Bilevel.simulate(sim_data,null_ctrl!,x0,N)

In [3]:
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf));
# open(mvis)
IJuliaCell(mvis)

WebIO.Scope("scope-880fa889-9a4a-4652-a9b7-ebec90887a7d", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :iframe), Any[], Dict{Symbol,Any}(:attributes=>Dict{String,Any}("height"=>"100%","scrolling"=>"no","src"=>"javascript:void(0)","frameborder"=>0),:id=>"ifr",:style=>Dict("width"=>"100%","flexGrow"=>"1")), 0)], Dict{Symbol,Any}(:style=>Dict("flexDirection"=>"column","height"=>"100%","minHeight"=>"400px","display"=>"flex","width"=>"100%","overflow"=>"hidden")), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}(), Set(String[]), nothing, Any[], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function () {\n    var frame = this.dom.querySelector(\"#ifr\");\n    var doc = frame.contentDocument\n    var win = frame.contentWindow\n\n    // Ensure that the iframe's baseURI matches the baseURI of the\n    // outer document. This is necessary to resolve\n    // https://github.com/JuliaGizmos/WebIO.jl/issues/167\n    var base = doc.createElement(\"base\");\n    base.setAttribute(\"href\", document.baseURI);\n    doc.head.appendChild(base);\n\n    var webio = doc.createElement(\"script\")\n    webio.src = \"/assetserver/b464848323e0a4c9a3cfe11f8edca2932188b6d9-assets/webio/dist/bundle.js\"\n    var parent = window\n\n    function resizeIframe() {\n        doc.body.style.padding = '0'\n        doc.body.style.margin = '0'\n        doc.documentElement.height = '100%'\n        doc.body.height = '100%'\n    }\n\n    webio.onload = function () {\n        win.WebIO.sendCallback = parent.WebIO.sendCallback; // Share stuff\n        win.WebIO.scopes = parent.WebIO.scopes\n        win.WebIO.obsscopes = parent.WebIO.obsscopes\n        win.WebIO._connected = true\n        doc.body.innerHTML = \"<html><body>\" + \"<div class='display:none'></div><unsafe-script style='display:none'>\\nWebIO.mount(this.previousSibling,{&quot;props&quot;:{},&quot;nodeType&quot;:&quot;Scope&quot;,&quot;type&quot;:&quot;node&quot;,&quot;instanceArgs&quot;:{&quot;imports&quot;:{&quot;data&quot;:[{&quot;name&quot;:null,&quot;type&quot;:&quot;js&quot;,&quot;url&quot;:&quot;/assetserver/cff5eb7af79db7e6d407febf164549a9fdb258cd-main.min.js&quot;}],&quot;type&quot;:&quot;async_block&quot;},&quot;id&quot;:&quot;scope-e0bd907a-fad0-4bb9-a0ee-67e52fad0281&quot;,&quot;handlers&quot;:{&quot;meshcat-command&quot;:[(function (val){return this.viewer.handle_command_message({&quot;data&quot;:val})})],&quot;_promises&quot;:{&quot;importsLoaded&quot;:[(function (mc){var element=this.dom.children[0]; this.viewer=new mc.Viewer(element); WebIO.setval({&quot;name&quot;:&quot;meshcat-request&quot;,&quot;scope&quot;:&quot;scope-e0bd907a-fad0-4bb9-a0ee-67e52fad0281&quot;,&quot;id&quot;:&quot;ob_02&quot;,&quot;type&quot;:&quot;observable&quot;},String(Date.now())); window.document.body.style.margin=&quot;0&quot;; return window.meshcat_viewer=this.viewer})]}},&quot;systemjs_options&quot;:null,&quot;observables&quot;:{&quot;meshcat-command&quot;:{&quot;sync&quot;:false,&quot;id&quot;:&quot;ob_01&quot;,&quot;value&quot;:[131,166,109,97,116,114,105,120,199,64,23,0,0,128,63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,128,63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,128,63,0,0,0,0,0,0,0,0,0,0,0,0,7,206,153,189,0,0,128,63,164,116,121,112,101,173,115,101,116,95,116,114,97,110,115,102,111,114,109,164,112,97,116,104,218,0,82,47,109,101,115,104,99,97,116,47,119,111,114,108,100,47,102,108,111,111,114,47,98,111,100,121,47,98,97,99,107,95,114,105,103,104,116,95,104,105,112,47,98,97,99,107,95,114,105,103,104,116,95,117,112,112,101,114,95,108,101,103,47,98,97,99,107,95,114,105,103,104,116,95,108,111,119,101,114,95,108,101,103]},&quot;meshcat-controls&quot;:{&quot;sync&quot;:true,&quot;id&quot;:&quot;ob_03&quot;,&quot;value&quot;:[]},&quot;meshcat-request&quot;:{&quot;sync&quot;:true,&quot;id&quot;:&quot;ob_02&quot;,&quot;value&quot;:&quot;&quot;}}},&quot;children&quot;:[{&quot;props&quot;:{&quot;className&quot;:&quot;meshcat-viewer&quot;,&quot;style&quo

In [ ]:
using Pkg
Pkg.status()

In [ ]:
setanimation!(mvis, traj[6], traj[1])

In [4]:
widget = manipulate!(mvis)

WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Observable{Any} with 1 listeners. Value:
nothing], Dict{Symbol,Any}(), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-60477376-ae8e-431d-a853-b657c3577bb1", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["floor_to_body.q1"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row-left"), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>629,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-center"), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_val")), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-right"), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row interact-widget"), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"index"=>(Observable{Any} with 2 listeners. Value:
315, nothing)), Set(String[]), nothing, Any["knockout"=>"/home/blandry/.julia/packages/Knockout/JE2Yq/src/../assets/knockout.js", "knockout_punches"=>"/home/blandry/.julia/packages/Knockout/JE2Yq/src/../assets/knockout_punches.js", "/home/blandry/.julia/packages/InteractBase/R8JgH/src/../assets/style.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"-3.14159\",\"-3.13159\",\"-3.12158\",\"-3.11158\",\"-3.10157\",\"-3.09157\",\"-3.08156\",\"-3.07156\",\"-3.06155\",\"-3.05155\",\"-3.04154\",\"-3.03154\",\"-3.02153\",\"-3.01153\",\"-3.00152\",\"-2.99152\",\"-2.98151\",\"-2.97151\",\"-2.9615\",\"-2.9515\",\"-2.94149\",\"-2.93149\",\"-2.92148\",\"-2.91148\",\"-2.90147\",\"-2.89147\",\"-2.88146\",\"-2.87146\",\"-2.86145\",\"-2.85145\",\"-2.84144\",\"-2.83144\",\"-2.82143\",\"-2.81143\",\"-2.80142\",\"-2.79142\",\"-2.78141\",\"-2.7714\",\"-2.7614\",\"-2.75139\",\"-2.74139\",\"-2.73138\",\"-2.72138\",\"-2.71137\",\"-2.70137\",\"-2.69136\",\"-2.68136\",\"-2.67135\",\"-2.66135\",\"-2.65134\",\"-2.64134\",\"-2.63133\",\"-2.62133\",\"-2.61132\",\"-2.60132\",\"-2.59131\",\"-2.58131\",\"-2.5713\",\"-2.5613\",\"-2.55129\",\"-2.54129\",\"-2.53128\",\"-2.52128\",\"-2.51127\",\"-2.50127\",\"-2.49126\",\"-2.48126\",\"-2.47125\",\"-2.46125\",\"-2.45124\",\"-2.44124\",\"-2.43123\",\"-2.42123\",\"-2.41122\",\"-2.40122\",\"-2.39121\",\"-2.38121\",\"-2.3712\",\"-2.3612\",\"-2.35119\",\"-2.34119\",\"-2.33118\",\"-2.32118\",\"-2.31117\",\"-2.30117\",\"-2.29116\",\"-2.28116\",\"-2.27115\",

MethodError: no method matching size(::SArray{Tuple{3},Float64,1,3})
The applicable method may be too new: running in world age 25581, while current world is 25982.
Closest candidates are:
  size(::Union{Adjoint{T,#s15} where #s15<:Union{StaticArray{Tuple{#s12},T,1} where #s12, StaticArray{Tuple{#s13,#s14},T,2} where #s14 where #s13}, Diagonal{T,#s18} where #s18<:(StaticArray{Tuple{#s19},T,1} where #s19), Hermitian{T,#s17} where #s17<:(StaticArray{Tuple{T,M},T1,2} where T1 where M), Symmetric{T,#s16} where #s16<:(StaticArray{Tuple{T,M},T1,2} where T1 where M), Transpose{T,#s12} where #s12<:Union{StaticArray{Tuple{#s12},T,1} where #s12, StaticArray{Tuple{#s13,#s14},T,2} where #s14 where #s13}, StaticArray{Tuple{#s19},T,1} where #s19, StaticArray{Tuple{#s14,#s13},T,2} where #s13 where #s14, StaticArray{#s19,T,N} where N where #s19<:Tuple} where T) at /home/blandry/.julia/packages/StaticArrays/VyRz3/src/abstractarray.jl:9 (method too new to be called from this world context.)
  size(::Uni

In [ ]:
q_nom = [0., -.175, 0., 0., 0., 0., 0., .036, 0., 0., 0., .00758, 0., .002445, 0.] 
q_min = [-5., -.25, -2*pi, -.6, -.6, -.6, -.6, -3.5, -3.5, -3.5, -3.5, -3.1, -3.1, -3.1, -3.1]
q_max = [5., -.1, 2*pi, .6, .6, .6, .6, 2.4, 2.4, 2.4, 2.4, 1., 1., 1., 1.]

N = 20
Δt = .1

function add_prob_constraints!(sim_data)
    vs = sim_data.vs

    add_eq!(sim_data, :cq1, num_positions(mechanism), x -> vs(x, :q1) - q_nom)
    add_eq!(sim_data, :cv1, num_velocities(mechanism), x -> vs(x, :v1))
    
    # final position
    add_eq!(sim_data, :cqN, 2, x -> vs(x, Symbol("q", N))[1:2] - [.1, -.175])
end

function add_prob_obj!(sim_data)
    for n = 1:N
        add_obj!(sim_data, Symbol("u", n), x -> sim_data.vs(x, Symbol("u", n))' * sim_data.vs(x, Symbol("u", n)))
    end
end

function get_prob_limits(sim_data)
    x_min = -1e19*ones(sim_data.vs.num_vars)
    x_max = 1e19*ones(sim_data.vs.num_vars)
    
    # joint limits
    add_box_con_snopt!(x_min, x_max, sim_data, :q, q_min, q_max, 1:N)
    add_box_con_snopt!(x_min, x_max, sim_data, :h, [.5 * Δt], [2. * Δt], 1:N-1)
    
    # torques limits
    u_limit = .5
    add_box_con_snopt!(x_min, x_max, sim_data, :u, -u_limit * ones(num_velocities(mechanism)), u_limit * ones(num_velocities(mechanism)), 1:N-1)
    
    x_min, x_max
end

function get_prob_init(sim_data)
    x0 = zeros(sim_data.vs.num_vars)
    for n = 1:N
        x0[sim_data.vs(Symbol("q", n))] .= q_nom
    end
    
    x0
end

function setup_prob!(sim_data)
    add_prob_constraints!(sim_data)
    add_prob_obj!(sim_data)
    x_min, x_max = get_prob_limits(sim_data)
    x0 = get_prob_init(sim_data)
    
    x0, x_min, x_max
end

In [ ]:
sim_data = get_trajopt_data_indirect(mechanism,env,Δt,N,relax_comp=false)
x0, x_min, x_max = setup_prob!(sim_data)
traj_indirect = Bilevel.trajopt(sim_data, x0=x0, x_min=x_min, x_max=x_max, verbose=1, opt_tol=1e-3, major_feas=1e-3, minor_feas=1e-3);

In [ ]:
setanimation!(mvis, traj_indirect[7], traj_indirect[1])

In [ ]:
sim_data = get_trajopt_data_semidirect(mechanism,env,Δt,N,relax_comp=false)
x0, x_min, x_max = setup_prob!(sim_data)
traj_semidirect = Bilevel.trajopt(sim_data, x0=x0, x_min=x_min, x_max=x_max, verbose=1, opt_tol=1e-3, major_feas=1e-3, minor_feas=1e-3);

In [ ]:
setanimation!(mvis, traj_semidirect[7], traj_semidirect[1])

In [ ]:
traj_semidirect[1]

## Panda arm